In [1]:
import cv2
import os
import numpy as np 
import keras
from tensorflow import keras
import matplotlib.pyplot as plt
import glob
import tensorflow as tf


2023-05-19 12:23:20.091489: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-19 12:23:20.093082: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-19 12:23:20.125744: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-19 12:23:20.126314: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-19 12:23:20.693331: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
# Primero, bajamos los datos de entrenamiento
keras.utils.get_file(fname="simpsons_train.tar.gz", 
                     origin="https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219337&authkey=AMzI92bJPx8Sd60")

# Descomprimimos el archivo
!tar -xzf /root/.keras/datasets/simpsons_train.tar.gz -C /root/.keras/datasets


# Hacemos lo mismo con los datos de test
keras.utils.get_file(fname="simpsons_test.tar.gz", 
                     origin="https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219341&authkey=ANnjK3Uq1FhuAe8")
!tar -xzf /root/.keras/datasets/simpsons_test.tar.gz -C /root/.keras/datasets

tar (child): /root/.keras/datasets/simpsons_train.tar.gz: No se puede efectuar open: Permiso denegado
tar (child): Error is not recoverable: exiting now

gzip: stdin: unexpected end of file
tar: Child returned status 2
tar: Error is not recoverable: exiting now
tar (child): /root/.keras/datasets/simpsons_test.tar.gz: No se puede efectuar open: Permiso denegado
tar (child): Error is not recoverable: exiting now

gzip: stdin: unexpected end of file
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [3]:
# Esta variable contiene un mapeo de número de clase a personaje.
# Utilizamos sólo los 18 personajes del dataset que tienen más imágenes.
MAP_CHARACTERS = {
    0: 'abraham_grampa_simpson', 1: 'apu_nahasapeemapetilon', 2: 'bart_simpson',
    3: 'charles_montgomery_burns', 4: 'chief_wiggum', 5: 'comic_book_guy', 6: 'edna_krabappel', 
    7: 'homer_simpson', 8: 'kent_brockman', 9: 'krusty_the_clown', 10: 'lisa_simpson', 
    11: 'marge_simpson', 12: 'milhouse_van_houten', 13: 'moe_szyslak', 
    14: 'ned_flanders', 15: 'nelson_muntz', 16: 'principal_skinner', 17: 'sideshow_bob'
}

# Vamos a standarizar todas las imágenes a tamaño 64x64
IMG_SIZE = 64

In [4]:
def load_train_set(dirname, map_characters, verbose=True):
    """Esta función carga los datos de training en imágenes.
    
    Como las imágenes tienen tamaños distintas, utilizamos la librería opencv
    para hacer un resize y adaptarlas todas a tamaño IMG_SIZE x IMG_SIZE.
    
    Args:
        dirname: directorio completo del que leer los datos
        map_characters: variable de mapeo entre labels y personajes
        verbose: si es True, muestra información de las imágenes cargadas
     
    Returns:
        X, y: X es un array con todas las imágenes cargadas con tamaño
                IMG_SIZE x IMG_SIZE
              y es un array con las labels de correspondientes a cada imagen
    """
    X_train = []
    y_train = []
    for label, character in map_characters.items():        
        files = os.listdir(os.path.join(dirname, character))
        images = [file for file in files if file.endswith("jpg")]
        if verbose:
          print("Leyendo {} imágenes encontradas de {}".format(len(images), character))
        for image_name in images:
            image = cv2.imread(os.path.join(dirname, character, image_name))
            X_train.append(cv2.resize(image,(IMG_SIZE, IMG_SIZE)))
            y_train.append(label)
    return np.array(X_train), np.array(y_train)

In [5]:
def load_test_set(dirname, map_characters, verbose=True):
    """Esta función funciona de manera equivalente a la función load_train_set
    pero cargando los datos de test."""
    X_test = []
    y_test = []
    reverse_dict = {v: k for k, v in map_characters.items()}
    for filename in glob.glob(dirname + '/*.*'):
        char_name = "_".join(filename.split('/')[-1].split('_')[:-1])
        if char_name in reverse_dict:
            image = cv2.imread(filename)
            image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
            X_test.append(image)
            y_test.append(reverse_dict[char_name])
    if verbose:
        print("Leídas {} imágenes de test".format(len(X_test)))
    return np.array(X_test), np.array(y_test)


In [6]:
# Cargamos los datos. Si no estás trabajando en colab, cambia los paths por
# los de los ficheros donde hayas descargado los datos.
DATASET_TRAIN_PATH_COLAB = "/root/.keras/datasets/simpsons"
DATASET_TEST_PATH_COLAB = "/root/.keras/datasets/simpsons_testset"

X, y = load_train_set(DATASET_TRAIN_PATH_COLAB, MAP_CHARACTERS)
X_t, y_t = load_test_set(DATASET_TEST_PATH_COLAB, MAP_CHARACTERS)

PermissionError: [Errno 13] Permission denied: '/root/.keras/datasets/simpsons/abraham_grampa_simpson'

In [ ]:
# Vamos a barajar aleatoriamente los datos. Esto es importante ya que si no
# lo hacemos y, por ejemplo, cogemos el 20% de los datos finales como validation
# set, estaremos utilizando solo un pequeño número de personajes, ya que
# las imágenes se leen secuencialmente personaje a personaje.
perm = np.random.permutation(len(X))
X, y = X[perm], y[perm]

In [ ]:
plt.imshow(X_t[2]) # recordad que siempre es preferible trabajar en blanco y negro


In [ ]:
print(f'Shape X: {X.shape}')
print(f'Shape y: {y.shape}')
print(f'Shape X_t: {X_t.shape}')
print(f'Shape y_t: {y_t.shape}')

# CAMBIAR A BLANCO Y NEGRO

In [ ]:
# Convertir imagenes de entrenamiento a blanco y negro

X_grayscale = np.empty((X.shape[0], X.shape[1], X.shape[2]), dtype=np.uint8)

for i in range(X.shape[0]):
    X_grayscale[i] = cv2.cvtColor(X[i], cv2.COLOR_BGR2GRAY)

# Convertir imagenes de test a blanco y negro

X_t_grayscale = np.empty((X_t.shape[0], X_t.shape[1], X_t.shape[2]), dtype=np.uint8)

for i in range(X_t.shape[0]):
    X_t_grayscale[i] = cv2.cvtColor(X_t[i], cv2.COLOR_BGR2GRAY)




In [ ]:
plt.imshow(X_t_grayscale[2])

In [ ]:
print(f'Shape X_grayscale: {X_grayscale.shape}')
print(f'Shape X_t_grayscale: {X_t_grayscale.shape}')


In [ ]:
X_grayscale_norm = X_grayscale.astype('float32') / 255.
X_t_grayscale_norm = X_t_grayscale.astype('float32') / 255.



In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_grayscale_norm, y,test_size=0.2, random_state=0)

In [ ]:
print(f'Shape X_train: {X_train.shape}')
print(f'Shape y_train: {y_train.shape}')
print(f'Shape X_valid: {X_valid.shape}')
print(f'Shape y_valid: {y_valid.shape}')

Ejercicio

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout 
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np
from keras.callbacks import ModelCheckpoint, EarlyStopping



In [ ]:
# y_train_cat = to_categorical(y_train, num_classes=18)
# y_valid_cat = to_categorical(y_valid, num_classes=18)
# y_t_cat = to_categorical(y_t, num_classes=18)


In [ ]:
# print(f'Shape y_train_cat: {y_train_cat.shape}')
# print(f'Shape y_valid_cat: {y_valid_cat.shape}')
# print(f'Shape y_t_cat: {y_t_cat.shape}')


**Comentarios del modelo:**   

*   Ha mejorado con 3 capas conv 
*   Dropout 0.5 mejor que 0.2 
*   Con adamax parece que va algo mejor.
*   Dense(128, activation='relu') ->80%
*   batch_size = 64 -> 0.8136










In [ ]:
from keras.callbacks import EarlyStopping, Callback

paciencia = 20

# EarlyStop si durante x epoch no se consigue ninguna mejora en val_accuracy
early_stopping = EarlyStopping(monitor='val_accuracy', patience=paciencia)

# Imprimir mensaje indicando que no se han conseguido mejoras durante x epoch.
class ImprimirMensajeCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if self.model.stop_training:
            print(f"El entrenamiento ha sido detenido, val_accuracy no ha mejorado durante {paciencia} epoch.")

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(256, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(256, activation='relu'),
    Dense(18, activation='softmax')
])

# model.summary()

model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train,
        epochs=200,
        validation_data=(X_valid, y_valid),
        verbose=1, 
        batch_size=256,
        callbacks=[early_stopping, ImprimirMensajeCallback()])



In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:
    # score = model.score(X_test, y_test)
    # pred = model.predict(X_test)
    # confusionmatrix = confusion_matrix(y_test,pred)
    # classificationreport = classification_report(y_test, pred)

    # print(f"Score: {score} \n")
    # print("Matriz de confusión:")
    # print(confusionmatrix)
    # print("\nClassification Report: ")
    # print(classificationreport) 

In [ ]:
# Evaluar el modelo en el conjunto de datos de prueba
test_loss, test_acc = model.evaluate(X_t_grayscale_norm, y_t)

# Mostrar la precisión y la pérdida del modelo en el conjunto de datos de test
print(f'\n El accuracy obtenido es: {round(test_acc, 5)}')
print(f'\n El loss: {round(test_loss, 5)}')

# **Callbacks**

In [ ]:
from keras.callbacks import EarlyStopping, Callback
import matplotlib.pyplot as plt

paciencia = 20

# EarlyStop si durante x epoch no se consigue ninguna mejora en val_accuracy
early_stopping = EarlyStopping(monitor='val_accuracy', patience=paciencia)

# Imprimir mensaje indicando que no se han conseguido mejoras durante x epoch.
class ImprimirMensajeCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if self.model.stop_training:
            print(f"El entrenamiento ha sido detenido, val_accuracy no ha mejorado durante {paciencia} epoch.")

# Modelo CNN definitivo

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(256, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(256, activation='relu'),
    Dense(18, activation='softmax')
])

# Pruebas CNN BATCH SIZE [32,64,128,256,512]



In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adamax', metrics=['accuracy'])

for i in [32,64,128,256,512]:
  print('\n')
  print('-------------------------------------')
  print('-------------------------------------')
  print(f'Prueba con Batch Size: {i}')

  history = model.fit(X_train, y_train,
          epochs=200,
          validation_data=(X_valid, y_valid),
          verbose=0, 
          batch_size = i,
          callbacks=[early_stopping, ImprimirMensajeCallback()])
  


  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(acc))

  plt.plot(epochs, acc, 'r', label='Training accuracy')
  plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
  plt.title('Training and validation accuracy')
  plt.legend(loc=0)
  plt.figure()

  plt.show()

  # Evaluar el modelo en el conjunto de datos de prueba
  print(f'\n')
  print(f'Métricas con dataset de validación:')

  valid_loss, valid_acc = model.evaluate(X_valid, y_valid)

  print(f'\n')
  print(f'Métricas con dataset de test:')

  test_loss, test_acc = model.evaluate(X_t_grayscale_norm, y_t)

  print(f'\n')

# Conclusión: Todas se comportan parecido




# **PRUEBAS CNN OPTIMIZER ["sgd", "rmsprop", "adam", "adagrad", "adadelta", "adamax", "nadam", "ftrl"]** 

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(256, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(256, activation='relu'),
    Dense(18, activation='softmax')
])

# model.summary()

# Lista de optimizadores que se van a utilizar durante las pruebas
lista_optimizadores = ["sgd", "rmsprop", "adam", "adagrad", "adadelta", "adamax", "nadam", "ftrl"]

for optimizador in lista_optimizadores:
  print('\n')
  print('-------------------------------------')
  print('-------------------------------------')
  print('\n')
  print(f' Pruebas con optimizador: {optimizador}')


  model.compile(loss = 'sparse_categorical_crossentropy', optimizer=optimizador, metrics=['accuracy'])

  history = model.fit(X_train, y_train,
          epochs=200,
          validation_data=(X_valid, y_valid),
          verbose=0, 
          batch_size = 32,
          callbacks=[early_stopping, ImprimirMensajeCallback()])

  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(acc))

  plt.plot(epochs, acc, 'r', label='Training accuracy')
  plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
  plt.title('Training and validation accuracy')
  plt.legend(loc=0)
  plt.figure()

  plt.show()

  # Evaluar el modelo en el conjunto de datos de prueba
  print(f'\n')
  print(f'Métricas con dataset de validación:')

  valid_loss, valid_acc = model.evaluate(X_valid, y_valid)

  print(f'\n')
  print(f'Métricas con dataset de test:')

  test_loss, test_acc = model.evaluate(X_t_grayscale_norm, y_t)

  print(f'\n')

# ANN (Con hiperparámetros seleccionados en CNN)

Función para complilar, fit y evaluar el modelo.   
Se muestra solo una evaluación con dataset de validadación

In [ ]:
def compile_fit_evaluate(model):
  model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adamax', metrics=['accuracy'])

  history = model.fit(X_train, y_train,
          epochs=200,
          validation_data=(X_valid, y_valid),
          verbose=0, 
          batch_size = 32,
          callbacks=[early_stopping, ImprimirMensajeCallback()])

  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(acc))

  plt.plot(epochs, acc, 'r', label='Training accuracy')
  plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
  plt.title('Training and validation accuracy')
  plt.legend(loc=0)
  plt.figure()

  plt.show()

  # Evaluar el modelo en el conjunto de datos de prueba
  print(f'\n')
  print(f'Métricas con dataset de validación:')

  valid_loss, valid_acc = model.evaluate(X_valid, y_valid)

  print(f'\n')
  print(f'Métricas con dataset de test:')

  test_loss, test_acc = model.evaluate(X_t_grayscale_norm, y_t)

  print(f'\n')

In [ ]:
model = Sequential([
    Flatten(input_shape=[64,64]),
    Dense(128, activation='relu'),
    Dense(18, activation='softmax')
])
compile_fit_evaluate(model)


In [ ]:
model = Sequential([
    Flatten(input_shape=[64,64]),
    Dense(128, activation='relu'),
    Dense(256, activation='relu'),
    Dense(18, activation='softmax')
])
compile_fit_evaluate(model)

In [ ]:
model = Sequential([
    Flatten(input_shape=[64,64]),
    Dense(128, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(18, activation='softmax')
])
compile_fit_evaluate(model)

In [ ]:
model = Sequential([
    Flatten(input_shape=[64,64]),
    Dense(128, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(512, activation='relu'),
    Dense(18, activation='softmax')
])
compile_fit_evaluate(model)

In [ ]:
model = Sequential([
    Flatten(input_shape=[64,64]),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(18, activation='softmax')
])
compile_fit_evaluate(model)

Comentar que con los mismos hiperparámetros los resultados son peores.   
Hace falta ajustarlos